### 1. Обучить любую модель классификации на датасете IRIS до применения PCA (2 компоненты) и после него. Сравнить качество классификации по отложенной выборке.

In [1]:
%matplotlib inline
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [3]:
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [4]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [5]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
            
        # найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction        

In [6]:
class Tree:
    
    # параметры для останова
    def __init__(self,
                 max_tree_depth_stop=np.inf,
                 max_leaf_num_stop=np.inf,
                 min_leaf_object_stop=1):
        
        self.max_depth = max_tree_depth_stop
        self.nodes = []
        self.leaves = []
        self.depth = 0
        self.max_leaves = max_leaf_num_stop
        self.min_objects = min_leaf_object_stop
        self.tree = None
        
    # Расчет критерия Джини
    def gini(self, labels):
        # подсчет количества объектов разных классов
        classes = {}
        for label in labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1

        # расчет критерия
        impurity = 1
        for label in classes:
            p = classes[label] / len(labels)
            impurity -= p ** 2

        return impurity
    
    # Расчет прироста
    def gain(self, left_labels, right_labels, root_gini):

        # доля выборки, ушедшая в левое поддерево
        p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])

        return root_gini - p * self.gini(left_labels) - (1 - p) * self.gini(right_labels)
    
    # Разбиение датасета в узле
    def split(self, data, labels, column_index, t):

        left = np.where(data[:, column_index] <= t)
        right = np.where(data[:, column_index] > t)

        true_data = data[left]
        false_data = data[right]

        true_labels = labels[left]
        false_labels = labels[right]

        return true_data, false_data, true_labels, false_labels
    
    # Нахождение наилучшего разбиения
    def find_best_split(self, data, labels):

        #  обозначим минимальное количество объектов в узле
        min_samples_leaf = 5

        root_gini = self.gini(labels)

        best_gain = 0
        best_t = None
        best_index = None

        n_features = data.shape[1]

        for index in range(n_features):
            # будем проверять только уникальные значения признака, исключая повторения
            t_values = np.unique(data[:, index])

            for t in t_values:
                true_data, false_data, true_labels, false_labels = self.split(data, labels, index, t)
                #  пропускаем разбиения, в которых в узле остается менее 5 объектов
                if len(true_data) < min_samples_leaf or len(false_data) < min_samples_leaf:
                    continue

                current_gain = self.gain(true_labels, false_labels, root_gini)

                #  выбираем порог, на котором получается максимальный прирост качества
                if current_gain > best_gain:
                    best_gain, best_t, best_index = current_gain, t, index

        return best_gain, best_t, best_index
    
    # Построение дерева с помощью рекурсивной функции
    def build_tree(self, data, labels):

        gain, t, index = self.find_best_split(data, labels)
 
        #  Базовый случай 2 - прекращаем рекурсию, когда достигли максимальной глубины дерева
        if self.depth > self.max_depth:
            self.leaves.append(Leaf(data, labels))
            return Leaf(data, labels)
        
        #  Базовый случай 3 - прекращаем рекурсию, когда достигли максимального количества листьев
        if len(self.leaves) >= self.max_leaves - 1 or self.depth >= self.max_leaves - 1:
            self.leaves.append(Leaf(data, labels))
            return Leaf(data, labels)
        
        #  Базовый случай 4 - прекращаем рекурсию, когда достигли минимального количества объектов в листе
        if len(data) <= self.min_objects:
            self.leaves.append(Leaf(data, labels))
            return Leaf(data, labels)
        
         #  Базовый случай 1 - прекращаем рекурсию, когда нет прироста в качества
        if gain == 0:
            self.leaves.append(Leaf(data, labels))
            return Leaf(data, labels)

        self.depth += 1
        
        true_data, false_data, true_labels, false_labels = self.split(data, 
                                                                      labels, 
                                                                      index, t)

        # Рекурсивно строим два поддерева
        true_branch = self.build_tree(true_data, true_labels)
        false_branch = self.build_tree(false_data, false_labels)

        # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
        self.nodes.append(Node(index, t, true_branch, false_branch))
        return Node(index, t, true_branch, false_branch)
    
    def classify_object(self, obj, node):

        if isinstance(node, Leaf):
            answer = node.prediction
            return answer

        if obj[node.index] <= node.t:
            return self.classify_object(obj, node.true_branch)
        else:
            return self.classify_object(obj, node.false_branch)
    
    def fit(self, data, labels):
        self.tree = self.build_tree(data, labels)
        return self
    
    def predict(self, data):
    
        classes = []
        for obj in data:
            prediction = self.classify_object(obj, self.tree)
            classes.append(prediction)
        return classes

In [7]:
tree_without_pca = Tree(max_tree_depth_stop=5)
tree_without_pca.fit(X_train, y_train)

In [8]:
accuracy_metric(y_train, tree_without_pca.predict(X_train))

94.28571428571428

In [9]:
accuracy_metric(y_test, tree_without_pca.predict(X_test))

97.77777777777777

In [10]:
eig_values, eig_vectors = np.linalg.eig(X.T @ X)
eig_values, eig_vectors

(array([9.20830507e+03, 3.15454317e+02, 1.19780429e+01, 3.55257020e+00]),
 array([[ 0.75110816,  0.2841749 ,  0.50215472,  0.32081425],
        [ 0.38008617,  0.5467445 , -0.67524332, -0.31725607],
        [ 0.51300886, -0.70866455, -0.05916621, -0.48074507],
        [ 0.16790754, -0.34367081, -0.53701625,  0.75187165]]))

In [11]:
eig_values_sum = eig_values.sum()
eig_values_importance = [(i / eig_values_sum) * 100 for i in sorted(eig_values, 
                                                                    reverse=True)]
eig_values_importance

[96.53029806531566,
 3.3068951313646853,
 0.12556535030291444,
 0.03724145301674341]

In [12]:
vector = np.hstack([eig_vectors.T[i].reshape(4, 1) for i in range(2)])
vector

array([[ 0.75110816,  0.2841749 ],
       [ 0.38008617,  0.5467445 ],
       [ 0.51300886, -0.70866455],
       [ 0.16790754, -0.34367081]])

In [13]:
Z = np.dot(X, vector)
Z

array([[ 5.91274714e+00,  2.30203322e+00],
       [ 5.57248242e+00,  1.97182599e+00],
       [ 5.44697714e+00,  2.09520636e+00],
       [ 5.43645948e+00,  1.87038151e+00],
       [ 5.87564494e+00,  2.32829018e+00],
       [ 6.47759822e+00,  2.32464996e+00],
       [ 5.51597520e+00,  2.07090423e+00],
       [ 5.85092859e+00,  2.14807482e+00],
       [ 5.15891972e+00,  1.77506408e+00],
       [ 5.64500117e+00,  1.99000106e+00],
       [ 6.26539771e+00,  2.42576813e+00],
       [ 5.75200785e+00,  2.02037338e+00],
       [ 5.48058085e+00,  1.97777558e+00],
       [ 4.95112411e+00,  2.04828749e+00],
       [ 6.52596417e+00,  2.91606081e+00],
       [ 6.79037199e+00,  2.82500759e+00],
       [ 6.27239468e+00,  2.60811578e+00],
       [ 5.92953789e+00,  2.26766614e+00],
       [ 6.64813130e+00,  2.38959506e+00],
       [ 6.09486463e+00,  2.36082303e+00],
       [ 6.25397363e+00,  2.12001187e+00],
       [ 6.07364677e+00,  2.27178150e+00],
       [ 5.36999813e+00,  2.49808604e+00],
       [ 6.

In [14]:
Z_train, Z_test, y_train, y_test = train_test_split(Z, y, test_size=0.3, random_state=42)

In [15]:
Z.shape

(150, 2)

In [16]:
X.shape

(150, 4)

In [17]:
tree_pca = Tree(max_tree_depth_stop=5)
tree_pca.fit(Z_train, y_train)

In [18]:
accuracy_metric(y_train, tree_pca.predict(Z_train))

98.09523809523809

In [19]:
accuracy_metric(y_test, tree_pca.predict(Z_test))

97.77777777777777

### 2. *Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции [numpy.linalg.svd()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html)

In [20]:
U, D, V = np.linalg.svd(X.T @ X)

In [21]:
U, D, V

(array([[-0.75110816, -0.2841749 ,  0.50215472,  0.32081425],
        [-0.38008617, -0.5467445 , -0.67524332, -0.31725607],
        [-0.51300886,  0.70866455, -0.05916621, -0.48074507],
        [-0.16790754,  0.34367081, -0.53701625,  0.75187165]]),
 array([9.20830507e+03, 3.15454317e+02, 1.19780429e+01, 3.55257020e+00]),
 array([[-0.75110816, -0.38008617, -0.51300886, -0.16790754],
        [-0.2841749 , -0.5467445 ,  0.70866455,  0.34367081],
        [ 0.50215472, -0.67524332, -0.05916621, -0.53701625],
        [ 0.32081425, -0.31725607, -0.48074507,  0.75187165]]))

In [22]:
D_sum = D.sum()
D_importance = [(i / D_sum) * 100 for i in D]
D_importance

[96.53029806531565,
 3.3068951313646853,
 0.12556535030291552,
 0.03724145301674215]

In [23]:
vector = np.hstack([V[i].reshape(4, 1) for i in range(2)])
Z = np.dot(X, vector)
Z = Z * -1

In [24]:
Z_train, Z_test, y_train, y_test = train_test_split(Z, y, test_size=0.3, random_state=42)

In [25]:
tree_pca_svd = Tree(max_tree_depth_stop=5)
tree_pca_svd.fit(Z_train, y_train)

In [26]:
accuracy_metric(y_train, tree_pca.predict(Z_train))

98.09523809523809

In [27]:
accuracy_metric(y_test, tree_pca.predict(Z_test))

97.77777777777777